In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from pyproj import CRS
import requests
import ssl


In [18]:
# Read GeoDataBase
ssl._create_default_https_context = ssl._create_unverified_context
# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs?request=GetFeature&typename=asuminen_ja_maankaytto:Vaestotietoruudukko_2018&outputformat=JSON'

data = gpd.read_file(url)

data.head(2)

,id,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80,geometry
0,Vaestotietoruudukko_2018.115,3342,108,45,11,23,6,7,26,17,8,6,4,MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...
1,Vaestotietoruudukko_2018.124,3503,273,35,35,24,52,62,40,26,25,9,0,MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...


In [28]:
# Define new geoid column
data['geoid'] = data.index.astype(str)
# Change columns name
data = data.rename(columns= {'asukkaita': 'pop18'})
# select columns to use
data = data[['geoid', 'pop18', 'geometry']]
# Check data
data.head(2)

,geoid,pop18,geometry
0,0,108,"MULTIPOLYGON Z (((24.57654 60.18042 0.00000, 2..."
1,1,273,"MULTIPOLYGON Z (((24.58102 60.18267 0.00000, 2..."


In [25]:
# Re-project the data to Web Mercator
data = data.to_crs(4326)

# check re-proj
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Create an interactive Croropleth Map from population Grid

In [43]:
# #======= Create Map instance ========== # #
m = folium.Map(location = [60.25, 24.8], 
               titles= 'cartodbpositron', 
               zoom_start=10, 
               min_zoom=9,
               max_zoom= 19,
               control_scale=True
              )

# #======= Add Layer to Map instance ========== # #
folium.Choropleth(
    geo_data=data,
    name='Population in 2018',
    data=data,
    columns=['geoid', 'pop18'],
    key_on= 'feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    #threshold_scale=[100, 250, 500, 1000, 2000],
    legend_name= 'Population in Helsinki'
).add_to(m)

# #======= Add ToolTips ========== # #

# define style function 
style_function= lambda x: {'color':'transparent', 'fillColor':'transparent', 'weight': 0}

# convert data into geojson
folium.features.GeoJson(data,
                       name='labels',
                       style_function= style_function,
                       tooltip=folium.features.GeoJsonTooltip(fields=['pop18'],
                                                             aliases=['Population'],
                                                             labels=True,
                                                             sticky=False
                                                             )
                       ).add_to(m)

# save Choropleth map
outfp = "maps\choropleth_ToolTips_map.html"
m.save(outfp)

# show map
m

Check in the link [plotly](https://plotly.com/python/mapbox-county-choropleth/) for more Choropleth maps 